In [239]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sept 20 16:43:08 2018
For: Starter_kit_cnns_pytorch
Author: Gaurav_Shrivastava 

"""


# Imports

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch
import numpy as np
from math import ceil
from statsmodels.stats.proportion import proportion_confint
from scipy.stats import norm 

In [240]:
# 2D Convolution Neural Network architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [241]:
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(784, 1024)
        self.fc2 = nn.Linear(1024, 784)
        self.fc3 = nn.Linear(784, 784)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return x#F.log_softmax(x, dim=1)

In [242]:
# Batch Training of model
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        noise_sd = torch.randn_like(data, device=device) * 0.25
        data = data + noise_sd
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [243]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [244]:
# def main():
    # Training settings
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = .01
momentum = 0.5
seed = 1
log_interval = 10

torch.manual_seed(seed)
device = torch.device("cpu")

kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for inputs,target in train_loader:
    break
# inputs = enumerate(next(train_loader))
print(torch.randn_like(inputs).shape)#* 2*torch.ones(1,28,28) )
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

torch.save(model,'trained_model.pth')



torch.Size([64, 1, 28, 28])
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.361161
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.315533
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296804
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.285949
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.253480
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.245948
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.162874
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.191080
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.096078
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.095489
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.995289
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.863797
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.686223
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.632440
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.768128
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.583664
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.440205
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.304146
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.257589
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.369363
T

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.458487
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.357913
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.338039
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.223922
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.525839
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.563139
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.371147
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.253809
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.370132
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.183261
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.154007
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.365907
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.344931
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.307467
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.342235
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.303546
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.173752
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.136192
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.678661
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.268413


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.259719
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.106918
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.460098
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.178450
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.198331
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.225120
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.232683
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.149130
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.248732
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.189672
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.254396
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.208957
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.264737
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.496284
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.165669
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.434096
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.357676
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.136154
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.309664
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.171769


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.187238
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.244204
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.307702
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.220010
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.168545
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.112917
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.141477
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.283631
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.161463
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.217116
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.146433
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.109830
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.067868
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.328358
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.468136
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.233416
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.140874
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.196450
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.366539
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.175468


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.148977
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.236438
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.405557
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.161859

Test set: Average loss: 0.0646, Accuracy: 9799/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.152773
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.202438
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.145075
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.144251
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.230926
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.083136
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.174904
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.116623
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.134817
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.190402
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.223560
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.122963
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.313172
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.132316
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.151842
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.086017
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.207519
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.264484
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.321717
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.136378
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.128999
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.144193
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.052675
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.111720
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.207878
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.125976
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.195605
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.294605
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.182388
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.268044
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.190851
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.161173
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.218165
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.216121
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.192621


C:\Users\yogau\anaconda3\lib\site-packages\torch\serialization.py:359: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "


In [245]:
def soft_argmax(x):
    beta = 2
    # x = torch.Tensor(np.array([[.2, .0, .81, .53, .8]]))
    a = torch.exp(beta*x)
    b = torch.sum(torch.exp(beta*x))
#     print(a,b)
    softmax = a/b
    max = torch.sum(softmax*x,1)
#     print(max)
    pos = x.size()
    
    softargmax = torch.sum(softmax*torch.arange(0,pos[1]))
    return softargmax
#     print(pos, softargmax)
#     mx = softargmax.int()
#     ans = softargmax.round()
# #     print(mx)
#     if ans>mx:
#         return mx +1.0
#     return mx + 0.0

#     print(softargmax.int())#,softmax*torch.arange(0,pos[1]))
    

In [255]:
class Smooth(object):
    """A smoothed classifier g """
    def __init__(self, base_classifier: torch.nn.Module, sigma, epsilon = 0.2):
        """
        :param base_classifier: maps from [batch x channel x height x width] to [batch x num_classes]
        :param num_classes:
        :param sigma: the noise level hyperparameter
        :param epsilon: hyperparameter for level of error
        """
        self.base_classifier = base_classifier
        self.sigma = sigma.view(1,28,28)
        self.target = None
        self.epsilon = epsilon

    def certify(self, x: torch.tensor, sigma, target, n: int, batch_size: int):
        """ Monte Carlo algorithm for certifying that g's prediction around x is constant within some L2 radius.
        With probability at least 1 - alpha, the class returned by this method will equal g(x), and g's prediction will
        robust within a L2 ball of radius R around x.

        :param x: the input [channel x height x width]
        :param n: the number of Monte Carlo samples to use for estimation
        :param batch_size: batch size to use when evaluating the base classifier
        :return: (predicted class, certified radius)
        """
        self.sigma = sigma.view(1,28,28)
        self.target = target +0.0
#         print(target.dtype)
        self.base_classifier.eval()
        # draw samples of f(x+ epsilon)
        cAHat, pABar = self.sample_noise(x,n,batch_size)
#         print(self.sigma)
        if pABar > 0.95:
            pABar = 0.95
        if pABar <0.5:
            radius = self.sigma* 0.0
        else:
            radius = self.sigma * norm.ppf(pABar)
#         print(pABar,norm.ppf(pABar))#,radius)
        return cAHat, radius


    def sample_noise(self, x: torch.tensor, num: int, batch_size) -> np.ndarray:
        """ Sample the base classifier's prediction under noisy corruptions of the input x.

        :param x: the input [channel x width x height]
        :param num: number of samples to collect
        :param batch_size:
        :return: an ndarray[int] of length num_classes containing the per-class counts
        """
        x = x.view(1,28,28)
#         with torch.no_grad():
        counts = 0
        for _ in range(ceil(num / batch_size)):
            this_batch_size = min(batch_size, num)
            num -= this_batch_size
            batch = x.repeat((this_batch_size, 1, 1, 1))
            noise = torch.randn_like(batch, device=device) * self.sigma
            scores = self.base_classifier(batch + noise).detach()#.argmax(1)
            predictions = []
            for i in range(len(scores)):
                arg_score =  soft_argmax(scores[i].view(1,-1))
                if torch.abs(arg_score - self.target)<self.epsilon:
                    counts +=1
                predictions.append(arg_score)
#             counts += self._count_arr(predictions, self.num_classes)
            return torch.stack(predictions).mean(), counts/len(scores)

In [259]:
# Batch Training of model
def train_sigma_network(sigma_model, certified_model, device, train_loader, optimizer, epoch, log_interval):
    sigma_model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        target = target + 0.0
        optimizer.zero_grad()
        sigma = torch.abs(sigma_model(data).view(data.shape))
        total_loss = 0
        pred_loss = 0
        R_loss = 0
        for i in range(len(data)):
            pred_score, R = certified_model.certify(data[i],sigma[i],target[i],100,1)
#             print(R.max(),R.min(), pred_score - target[i])
            pred_loss += 0.1*torch.abs(pred_score - target[i])#F.mse_loss(pred_score, target[i].int())
            R_loss -= R.mean()
#         print(R_loss,pred_loss)
        total_loss = pred_loss +R_loss
#         print()
        clip_value = 0.1
#         for p in sigma_model.parameters():
#             p.register_hook(lambda grad: torch.clamp(grad, -clip_value, clip_value))
        total_loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), total_loss.item()))
#         break

In [129]:
sigma_model = FCN().to(device)
optimizer = optim.SGD(sigma_model.parameters(), lr=0.0001, momentum=momentum)

In [ ]:
sigma_model = FCN().to(device)
optimizer = optim.SGD(sigma_model.parameters(), lr=0.0000001, momentum=momentum)
# print(torch.abs(sigma_model(inputs)).view(inputs.shape)[0].shape)
model.eval()
l1loss = nn.L1Loss()
certified_model = Smooth(model,torch.abs(sigma_model(inputs)).view(inputs.shape)[0])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

train_sigma_network(sigma_model,certified_model,device,train_loader,optimizer,epoch,log_interval)

Train Epoch: 10 [0/60000 (0%)]	Loss: -23.868286
Train Epoch: 10 [1000/60000 (2%)]	Loss: -23.588120
Train Epoch: 10 [2000/60000 (3%)]	Loss: -22.454565
Train Epoch: 10 [3000/60000 (5%)]	Loss: -23.845589
Train Epoch: 10 [4000/60000 (7%)]	Loss: -24.025673
Train Epoch: 10 [5000/60000 (8%)]	Loss: -23.100605
Train Epoch: 10 [6000/60000 (10%)]	Loss: -23.667347
Train Epoch: 10 [7000/60000 (12%)]	Loss: -24.931986
Train Epoch: 10 [8000/60000 (13%)]	Loss: -24.474319
Train Epoch: 10 [9000/60000 (15%)]	Loss: -21.893248
Train Epoch: 10 [10000/60000 (17%)]	Loss: -24.040825
Train Epoch: 10 [11000/60000 (18%)]	Loss: -23.246632
Train Epoch: 10 [12000/60000 (20%)]	Loss: -21.363678
Train Epoch: 10 [13000/60000 (22%)]	Loss: -24.060905
Train Epoch: 10 [14000/60000 (23%)]	Loss: -23.424728
Train Epoch: 10 [15000/60000 (25%)]	Loss: -23.385860
Train Epoch: 10 [16000/60000 (27%)]	Loss: -23.639082
Train Epoch: 10 [17000/60000 (28%)]	Loss: -22.568329
Train Epoch: 10 [18000/60000 (30%)]	Loss: -22.068335
Train Epoch:

In [237]:
import torchvision
import matplotlib.pyplot as plt
sigma = sigma_model(inputs).view(inputs.shape)
# print(sigma)
torchvision.utils.save_image(sigma,'visual_sigma.png')